# ImageCreator

File Name: ImageCreator.ipnyb

Description: Using DALL-E and ChatGPT APIs, this program creates Pillow composite images of randomly-generated pictures and captions from a text file of input prompts. The program is divided into 3 classes:

- OpenAI: Using an input text file and DALL-E/ChatGPT APIs, this class generates DALL-E output image and ChatGPT output text from a randomly-generated input prompt.

- Slide: This class concatenates DALL-E image and caption.

- Main: This class saves images to computer in a while loop.

Author: Adam Post
Date: 8/8/2023

In [2]:
# Imports relevant packages
import os
import numpy as np
import requests
import random
import string
import io
import re
import PIL
from PIL import Image, ImageDraw, ImageFont
import scipy
from scipy.stats import truncnorm

In [3]:
# Class that creates DALLE image and ChatGPT text
class OpenAI:
    
    # Stores .txt file into list of dictionaries
    with open('ai_prompts.txt', 'r', encoding="utf8") as ai_prompts:
        promptDict = {key.replace('\n', ''):value.replace('\n', '') for key, value in zip(ai_prompts, ai_prompts)}      
        ai_prompts.close()
    
    def __init__(self):
        with open('ai_prompts.txt', 'r', encoding="utf8") as ai_prompts:
            promptDict = {key.replace('\n', ''):value.replace('\n', '') for key, value in zip(ai_prompts, ai_prompts)}      
            ai_prompts.close()
        self.randPrompt = random.randint(0, len(promptDict) - 1)

    # Truncated normal distributions for max_tokens within ChatGPT()
    def truncated_normal(mean, sd, low, upp, size):
        return (truncnorm.rvs(a=(low - mean) / sd, b=(upp - mean) / sd, size=size)) * sd + mean
    
    # Creating bi-modal distributions for max_tokens and normal distribution for temp
    temp = truncated_normal(1.25, 0.5, 0, 2, 1)
    normal1 = np.round(truncated_normal(50, 5, 20, 100, 1))
    normal2 = np.round(truncated_normal(70, 10, 20, 100, 1))
    if(np.random.rand() > 0.5):
        useNorm = normal1
    else: useNorm = normal2

    # Definition returns full ChatGPT output body
    def ChatGPT(self):
        # ChatGPT call
        GPTInput = list(self.promptDict)[self.randPrompt]
        url = "https://api.openai.com/v1/chat/completions"
        headers = {
        "Content-Type": "application/json",
        "Authorization": "INSERT API KEY",
        }
        data = { 
        "model": "gpt-3.5-turbo",
        "messages": [{"role": "system", "content": GPTInput}],
        "max_tokens": int(self.useNorm[0]), # max_tokens is randomly generated per above
        "temperature": self.temp[0] # Temperature is randomly generated per above
        }
        response = requests.post(url, headers=headers, json=data)
        
        # Accessing GPT string, outputs unparsed text
        output = response.json()
        unparsedText = list(output.values())[4][0]['message']['content']
        
        # Parses text; removes sentences about being an AI language model
        cleaned_str = str()
        for sentence in unparsedText.split("(.)"):
            if not(re.search("ethical guidelines|tokens|As an AI language model|Task is not feasible|However, I can|I'm sorry, I cannot|I'm sorry, but| My purpose is|AI|maximum token limit| I'm sorry, but|helpful and informative", sentence, flags=re.IGNORECASE)):
                cleaned_str += sentence
                
        # Returns cleaned ChatGPT output string
        return cleaned_str
    
    # Definition returns DALL-E image
    def DALLE(self):
        # DALLE call
        DALLEInput = list(self.promptDict.values())[self.randPrompt]
        try:
            url = "https://api.openai.com/v1/images/generations"
            headers = {
            "Content-Type": "application/json",
            "Authorization": "INSERT API KEY",
            }
            data = { 
              "prompt": DALLEInput,
              "n": 1,
              "size":"1024x1024"
            }
            response = requests.post(url, headers=headers, json=data)
            output = response.json()
            url = output["data"][0]["url"]
        except:
            # Generates random image when original fails
            url = "https://api.openai.com/v1/images/generations"
            headers = {
            "Content-Type": "application/json",
            "Authorization": "INSERT API KEY",
            }
            data = { 
              "prompt": "beef, lobsters and toilets",
              "n": 1,
              "size":"1024x1024"
            }
            response = requests.post(url, headers=headers, json=data)
            output = response.json()
            url = output["data"][0]["url"]
            
        # Downloads and outputs image
        image_dir = 'C:\\Users\\ap625\\Documents\\AI Absurdism' # Sets file name and directory
        res = ''.join(random.choices(string.ascii_uppercase +
                                 string.digits, k=10))
        generated_image_name = str(res) + '.png'
        generated_image_filepath = os.path.join(image_dir, generated_image_name)
        generated_image = requests.get(url)  # Downloads the image
        return Image.open(io.BytesIO(generated_image.content))

In [4]:
# Class that combines DALLE image with ChatGPT caption into Pillow image
class Slide:
    
    # Wraps text
    def wrap_text(self, text, max_width, font):
        lines = []
        words = text.split(' ')
        
        current_line = ''
        for word in words:
            test_line = current_line + word + ' '
            line_width = font.getlength(test_line)
            if line_width <= max_width:
                current_line = test_line
            else:
                lines.append(current_line[:-1])
                current_line = word + ' '
    
        lines.append(current_line[:-1])
        return lines

    # Concatenates DALLE image with ChatGPT caption
    def get_concat_v(self, im1, im2):
        dst = Image.new('RGB', (im1.width, im1.height + im2.height))
        dst.paste(im1, (0, 0))
        dst.paste(im2, (0, im1.height))
        return dst

    def create_image(self):
        ai = OpenAI() # Calls OpenAI class
        
        # Random background color
        colorList = ['yellow', 'orange', 'lightgreen', 'lightblue', 'pink', 'aquamarine',
                 'tomato', 'lavender', 'deepskyblue', 'salmon', 'springgreen', 'mediumslateblue', 'fuchsia', 'beige']
        randColor = colorList[random.randint(0, len(colorList) - 1)]
        
        # Random font
        fontList = os.listdir('C:/Users/ap625/Documents/AI Absurdism/Fonts')
        randFont = fontList[random.randint(0, len(fontList) - 1)]
        fontDir = 'C:/Users/ap625/Documents/AI Absurdism/Fonts/' + randFont
        font = ImageFont.truetype(fontDir, 50, encoding = "unic")
        
        # Creates GPT text and wraps it
        GPTOutput = ai.ChatGPT()
        while GPTOutput == '':
            GPTOutput = ai.ChatGPT()
        wrapped_text = self.wrap_text(GPTOutput, 1024, font)
        
        # Creates text box background
        img = Image.new(size=(1024, 300), mode='RGB', color=randColor)
        draw = ImageDraw.Draw(img)
        
        # Finishes text wrapping
        line_spacing = 1
        y = (300 - int(50 * len(wrapped_text) * line_spacing)) // 2
     
        for line in wrapped_text:
            line_width = font.getlength(line)
            draw.text(((1024 - line_width) // 2, y), line, "black", font=font)
            y += int(50 * line_spacing)
        
        # Concatenates image with caption
        return self.get_concat_v(ai.DALLE(), img)

In [5]:
# Class to iterate saving pictures to local computer
class main:
    i = 1
    
    while i < 2: # Choose any number of images to generate
        
        try:
            slide = Slide() # Creates slide class
        except:
            slide = Slide() # Creates slide class when original call fails
        
        # Stores image file to computer
        res = ''.join(random.choices(string.ascii_uppercase +
                                     string.digits, k=12))
        generated_image_name = str(res) + '.jpg'
        direc = 'C:/Users/ap625/Documents/AI Absurdism/Output/' + generated_image_name
    
        # Creates image and saves to directory
        final_image = slide.create_image()
        final_image.save(direc)
        i = i + 1